<a href="https://colab.research.google.com/github/peaceful-1/peaceful/blob/main/Bible_webscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True) # The force_remount=True flag forces the unmounting and remounting of the drive

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
!pip install openpyxl

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def scrapeBibleExcel(book_names, chapters, version='RSV'):
    base_url = "https://www.bible.com/bible/2020/"

    # Create a list to hold all verses for a single DataFrame
    all_verses = []

    for book in book_names:
        for chapter in chapters:
            try:
                # Construct the URL based on the book name and chapter
                url = f"{base_url}{book.upper()}.{chapter}.{version}"
                print(f"Scraping: {url}")

                # Send a request to the constructed URL
                response = requests.get(url)
                if response.status_code != 200:
                    print(f"Failed to retrieve {book} {chapter}. Skipping...")
                    continue

                # Parse the HTML content using BeautifulSoup
                soup = BeautifulSoup(response.content, 'html.parser')

                # Extract the text content from the relevant div
                chapter_content = soup.find('div', class_='ChapterContent_verse__57FIw')
                if not chapter_content:
                    print(f"No content found for {book} {chapter}. Skipping...")
                    continue

                text = chapter_content.get_text()

                # Check for the specific message indicating unavailability
                if "This chapter is not available in this version. Please choose a different chapter or version." in text:
                    print(f"Chapter {book} {chapter} is not available in this version. Skipping...")
                    continue

                # Use regex to split the text into two parts: content before the first verse and the actual verses
                pre_verse = re.split(r'\d+', text, 1)[0].strip()  # Everything before the first number
                verses = re.findall(r'(\d+)([^\d]+)', text)  # Capturing verse numbers and corresponding text

                # Create a DataFrame for the verses
                df = pd.DataFrame(verses, columns=['Verse Number', 'Verse Text'])

                # Insert the pre-verse content at the top
                pre_verse_df = pd.DataFrame({'Verse Number': ['Pre-Verses'], 'Verse Text': [pre_verse]})
                df = pd.concat([pre_verse_df, df], ignore_index=True)

                # Add book and chapter columns
                df['Book'] = book.upper()
                df['Chapter'] = chapter
                df['Verse']: verses
                df['Text']: text
                df['Version']: version

                # Append the DataFrame for the current chapter to the list of all verses
                all_verses.append(df)

            except Exception as e:
                print(f"Error processing {book} {chapter}: {e}. Skipping to the next chapter.")
                continue

    # Concatenate all verses DataFrames into a single DataFrame
    final_df = pd.concat(all_verses, ignore_index=True) if all_verses else pd.DataFrame()

    # Save all verses to a single sheet in the Excel file
    final_df.to_excel('Bible_Verses.xlsx', index=False)

    print("Bible verses have been saved to 'Bible_Verses.xlsx'.")

# Define the list of book names and chapter range (1 to 150)
book_chapters = {
    "gen": 50, "exo": 40, "lev": 27, "num": 36, "deu": 34, "jos": 24, "jdg": 21, "rut": 4,
    "1sa": 31, "2sa": 24, "1ki": 22, "2ki": 25, "1ch": 29, "2ch": 36, "ezr": 10, "neh": 13,
    "est": 10, "job": 42, "psa": 150, "pro": 31, "ecc": 12, "sng": 8, "isa": 66, "jer": 52,
    "lam": 5, "ezk": 48, "dan": 12, "hos": 14, "jol": 3, "amo": 9, "oba": 1, "jon": 4,
    "mic": 7, "nam": 3, "hab": 3, "zep": 3, "hag": 2, "zec": 14, "mal": 4, "mat": 28, "mrk": 16,
    "luk": 24, "jhn": 21, "act": 28, "rom": 16, "1co": 16, "2co": 13, "gal": 6, "eph": 6,
    "php": 4, "col": 4, "1th": 5, "2th": 3, "1ti": 6, "2ti": 4, "tit": 3, "phm": 1, "heb": 13,
    "jas": 5, "1pe": 5, "2pe": 3, "1jo": 5, "2jn": 1, "3jn": 1, "jud": 1, "rev": 22

# Call the function
scrapeBibleExcel(book_names, chapters)



SyntaxError: '{' was never closed (<ipython-input-4-8ae15aa26fd2>, line 75)

In [ ]:
# Install required libraries
!pip install requests beautifulsoup4 pandas openpyxl

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Base URL for Bible.com
BASE_URL = "https://www.bible.com/bible/477/GEN.1.RV1885"

# Dictionary of Bible books with their exact chapter counts
BIBLE_BOOKS = {
    'gen': 50, 'exo': 40, 'lev': 27, 'num': 36, 'deu': 34, 'jos': 24, 'jdg': 21, 'rut': 4,
    '1sa': 31, '2sa': 24, '1ki': 22, '2ki': 25, '1ch': 29, '2ch': 36, 'ezr': 10, 'neh': 13,
    'est': 10, 'job': 42, 'psa': 150, 'pro': 31, 'ecc': 12, 'sng': 8, 'isa': 66, 'jer': 52,
    'lam': 5, 'ezk': 48, 'dan': 12, 'hos': 14, 'jol': 3, 'amo': 9, 'oba': 1, 'jon': 4,
    'mic': 7, 'nam': 3, 'hab': 3, 'zep': 3, 'hag': 2, 'zec': 14, 'mal': 4, 'mat': 28,
    'mrk': 16, 'luk': 24, 'jhn': 21, 'act': 28, 'rom': 16, '1co': 16, '2co': 13, 'gal': 6,
    'eph': 6, 'php': 4, 'col': 4, '1th': 5, '2th': 3, '1ti': 6, '2ti': 4, 'tit': 3, 'phm': 1,
    'heb': 13, 'jas': 5, '1pe': 5, '2pe': 3, '1jo': 5, '2jn': 1, '3jn': 1, 'jud': 1, 'rev': 22
}

def scrape_bible_verses(book_names, version='RSV'):
    # Create a list to hold all verses
    all_verses = []

    for book in book_names:
        # Get the exact chapter count for the book
        chapter_count = BIBLE_BOOKS.get(book.lower(), 0)
        if chapter_count == 0:
            print(f"Invalid book name: {book}. Skipping...")
            continue

        for chapter in range(1, chapter_count + 1):
            try:
                # Construct the URL based on the book name and chapter
                url = f"{BASE_URL}{book.upper()}.{chapter}.{version}"
                print(f"Scraping: {url}")

                # Send a request to the constructed URL
                response = requests.get(url)
                if response.status_code != 200:
                    print(f"Failed to retrieve {book} {chapter}. Skipping...")
                    continue

                # Parse the HTML content using BeautifulSoup
                soup = BeautifulSoup(response.content, 'html.parser')

                # Extract the text content from the relevant div
                chapter_content = soup.find('div', class_='ChapterContent_yv-bible-text__tqVMm')
                if not chapter_content:
                    print(f"No content found for {book} {chapter}. Skipping...")
                    continue

                # Get the text and clean it
                text = chapter_content.get_text(separator=" ")  # Use separator to handle line breaks
                text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces and line breaks

                # Check for the specific message indicating unavailability
                if "This chapter is not available in this version. Please choose a different chapter or version." in text:
                    print(f"Chapter {book} {chapter} is not available in this version. Skipping...")
                    continue

                # Use regex to split the text into verses
                verses = re.findall(r'(\d+)\s*([^\d]+)', text)  # Capturing verse numbers and corresponding text

                # Add each verse to the list
                for verse_num, verse_text in verses:
                    # Clean the verse text
                    verse_text = re.sub(r'\s+', ' ', verse_text).strip()  # Remove extra spaces
                    if verse_text:  # Only add non-empty verses
                        all_verses.append({
                            'Version': version,  # Include the Bible version
                            'Book': book.upper(),
                            'Chapter': chapter,
                            'Verse': verse_num.strip(),
                            'Text': verse_text
                        })

            except Exception as e:
                print(f"Error processing {book} {chapter}: {e}. Skipping to the next chapter.")
                continue

    return all_verses

# Define the list of book names
book_names = list(BIBLE_BOOKS.keys())

# Scrape the Bible verses
version = 'RSV'  # Specify the Bible version
final_data = scrape_bible_verses(book_names, version=version)

# Save data to DataFrame
df = pd.DataFrame(final_data, columns=["Version", "Book", "Chapter", "Verse", "Text"])

# Save the DataFrame to an Excel file
excel_file = "bible_scraped_data.xlsx"
df.to_excel(excel_file, index=False)

# Download the file in Google Colab
from google.colab import files
files.download(excel_file)

print("Scraping Complete. Data saved and downloaded as Excel.")


Scraping: https://www.bible.com/bible/477/GEN.1.RV1885GEN.1.RSV
Scraping: https://www.bible.com/bible/477/GEN.1.RV1885GEN.2.RSV
Scraping: https://www.bible.com/bible/477/GEN.1.RV1885GEN.3.RSV
Scraping: https://www.bible.com/bible/477/GEN.1.RV1885GEN.4.RSV
Scraping: https://www.bible.com/bible/477/GEN.1.RV1885GEN.5.RSV
Scraping: https://www.bible.com/bible/477/GEN.1.RV1885GEN.6.RSV
Scraping: https://www.bible.com/bible/477/GEN.1.RV1885GEN.7.RSV
Scraping: https://www.bible.com/bible/477/GEN.1.RV1885GEN.8.RSV
Scraping: https://www.bible.com/bible/477/GEN.1.RV1885GEN.9.RSV
Scraping: https://www.bible.com/bible/477/GEN.1.RV1885GEN.10.RSV
Scraping: https://www.bible.com/bible/477/GEN.1.RV1885GEN.11.RSV
Scraping: https://www.bible.com/bible/477/GEN.1.RV1885GEN.12.RSV
Scraping: https://www.bible.com/bible/477/GEN.1.RV1885GEN.13.RSV
Scraping: https://www.bible.com/bible/477/GEN.1.RV1885GEN.14.RSV
Scraping: https://www.bible.com/bible/477/GEN.1.RV1885GEN.15.RSV
Scraping: https://www.bible.com/bi

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Scraping Complete. Data saved and downloaded as Excel.


In [ ]:
# Install required libraries
!pip install requests beautifulsoup4 pandas openpyxl

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Base URL for Bible.com
BASE_URL = "https://www.bible.com/bible/3548/"

# Dictionary of Bible books with their exact chapter counts
BIBLE_BOOKS = {
    'gen': 50, 'exo': 40, 'lev': 27, 'num': 36, 'deu': 34, 'jos': 24, 'jdg': 21, 'rut': 4,
    '1sa': 31, '2sa': 24, '1ki': 22, '2ki': 25, '1ch': 29, '2ch': 36, 'ezr': 10, 'neh': 13,
    'est': 10, 'job': 42, 'psa': 150, 'pro': 31, 'ecc': 12, 'sng': 8, 'isa': 66, 'jer': 52,
    'lam': 5, 'ezk': 48, 'dan': 12, 'hos': 14, 'jol': 3, 'amo': 9, 'oba': 1, 'jon': 4,
    'mic': 7, 'nam': 3, 'hab': 3, 'zep': 3, 'hag': 2, 'zec': 14, 'mal': 4, 'mat': 28,
    'mrk': 16, 'luk': 24, 'jhn': 21, 'act': 28, 'rom': 16, '1co': 16, '2co': 13, 'gal': 6,
    'eph': 6, 'php': 4, 'col': 4, '1th': 5, '2th': 3, '1ti': 6, '2ti': 4, 'tit': 3, 'phm': 1,
    'heb': 13, 'jas': 5, '1pe': 5, '2pe': 3, '1jo': 5, '2jn': 1, '3jn': 1, 'jud': 1, 'rev': 22
}

def scrape_bible_verses(book_names, version='RSVCI'):
    # Create a list to hold all verses
    all_verses = []

    for book in book_names:
        # Get the exact chapter count for the book
        chapter_count = BIBLE_BOOKS.get(book.lower(), 0)
        if chapter_count == 0:
            print(f"Invalid book name: {book}. Skipping...")
            continue

        for chapter in range(1, chapter_count + 1):
            try:
                # Construct the URL based on the book name and chapter
                url = f"{BASE_URL}{book.upper()}.{chapter}.{version}"
                print(f"Scraping: {url}")

                # Send a request to the constructed URL
                response = requests.get(url)
                if response.status_code != 200:
                    print(f"Failed to retrieve {book} {chapter}. Skipping...")
                    continue

                # Parse the HTML content using BeautifulSoup
                soup = BeautifulSoup(response.content, 'html.parser')

                # Extract the text content from the relevant div
                chapter_content = soup.find('div', class_='ChapterContent_verse__57FIw')
                if not chapter_content:
                    print(f"No content found for {book} {chapter}. Skipping...")
                    continue

                # Get the text and clean it
                text = chapter_content.get_text(separator=" ")  # Use separator to handle line breaks
                text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces and line breaks

                # Check for the specific message indicating unavailability
                if "This chapter is not available in this version. Please choose a different chapter or version." in text:
                    print(f"Chapter {book} {chapter} is not available in this version. Skipping...")
                    continue

                # Use regex to split the text into verses
                verses = re.findall(r'(\d+)\s*([^\d]+)', text)  # Capturing verse numbers and corresponding text

                # Add each verse to the list
                for verse_num, verse_text in verses:
                    # Clean the verse text
                    verse_text = re.sub(r'\s+', ' ', verse_text).strip()  # Remove extra spaces
                    if verse_text:  # Only add non-empty verses
                        all_verses.append({
                            'Version': version,  # Include the Bible version
                            'Book': book.upper(),
                            'Chapter': chapter,
                            'Verse': verse_num.strip(),
                            'Text': verse_text
                        })

            except Exception as e:
                print(f"Error processing {book} {chapter}: {e}. Skipping to the next chapter.")
                continue

    return all_verses

# Define the list of book names
book_names = list(BIBLE_BOOKS.keys())

# Scrape the Bible verses
version = 'RSVCI'  # Specify the Bible version
final_data = scrape_bible_verses(book_names, version=version)

# Save data to DataFrame
df = pd.DataFrame(final_data, columns=["Version", "Book", "Chapter", "Verse", "Text"])

# Save the DataFrame to an Excel file
excel_file = "bible_scraped_data.xlsx"
df.to_excel(excel_file, index=False)

# Download the file in Google Colab
from google.colab import files
files.download(excel_file)

print("Scraping Complete. Data saved and downloaded as Excel.")

Scraping: https://www.bible.com/bible/3548/GEN.1.RSVCI
No content found for gen 1. Skipping...
Scraping: https://www.bible.com/bible/3548/GEN.2.RSVCI
No content found for gen 2. Skipping...
Scraping: https://www.bible.com/bible/3548/GEN.3.RSVCI
No content found for gen 3. Skipping...
Scraping: https://www.bible.com/bible/3548/GEN.4.RSVCI
No content found for gen 4. Skipping...
Scraping: https://www.bible.com/bible/3548/GEN.5.RSVCI
No content found for gen 5. Skipping...
Scraping: https://www.bible.com/bible/3548/GEN.6.RSVCI
No content found for gen 6. Skipping...
Scraping: https://www.bible.com/bible/3548/GEN.7.RSVCI
No content found for gen 7. Skipping...
Scraping: https://www.bible.com/bible/3548/GEN.8.RSVCI
No content found for gen 8. Skipping...
Scraping: https://www.bible.com/bible/3548/GEN.9.RSVCI
No content found for gen 9. Skipping...
Scraping: https://www.bible.com/bible/3548/GEN.10.RSVCI
No content found for gen 10. Skipping...
Scraping: https://www.bible.com/bible/3548/GEN.1

KeyboardInterrupt: 

In [ ]:
# Install required libraries
!pip install requests beautifulsoup4 pandas openpyxl

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

# Base URL for Bible.com
BASE_URL = "https://www.bible.com/bible/477/GEN.1.RV1885"

def scrape_bible_verses(book_names, chapters, version='RV1885'):
    # Create a list to hold all verses
    all_verses = []

    for book in book_names:
        for chapter in chapters:
            try:
                # Construct the URL based on the book name and chapter
                url = f"{BASE_URL}{book.upper()}.{chapter}.{version}"
                print(f"Scraping: {url}")

                # Send a request to the constructed URL
                response = requests.get(url)
                if response.status_code != 200:
                    print(f"Failed to retrieve {book} {chapter}. Skipping...")
                    continue

                # Parse the HTML content using BeautifulSoup
                soup = BeautifulSoup(response.content, 'html.parser')

                # Extract the text content from the relevant div
                chapter_content = soup.find('div', class_='ChapterContent_reader__Dt27r')
                if not chapter_content:
                    print(f"No content found for {book} {chapter}. Skipping...")
                    continue

                text = chapter_content.get_text()

                # Check for the specific message indicating unavailability
                if "This chapter is not available in this version. Please choose a different chapter or version." in text:
                    print(f"Chapter {book} {chapter} is not available in this version. Skipping...")
                    continue

                # Use regex to split the text into verses
                verses = re.findall(r'(\d+)([^\d]+)', text)  # Capturing verse numbers and corresponding text

                # Add each verse to the list
                for verse_num, verse_text in verses:
                    all_verses.append({
                        'Version' : version,
                        'Book': book.upper(),
                        'Chapter': chapter,
                        'Verse': verse_num.strip(),
                        'Text': verse_text.strip()
                    })

            except Exception as e:
                print(f"Error processing {book} {chapter}: {e}. Skipping to the next chapter.")
                continue

    return all_verses


# Dictionary of Bible books with their exact chapter counts
BIBLE_BOOKS = {
    'gen': 50, 'exo': 40, 'lev': 27, 'num': 36, 'deu': 34, 'jos': 24, 'jdg': 21, 'rut': 4,
    '1sa': 31, '2sa': 24, '1ki': 22, '2ki': 25, '1ch': 29, '2ch': 36, 'ezr': 10, 'neh': 13,
    'est': 10, 'job': 42, 'psa': 150, 'pro': 31, 'ecc': 12, 'sng': 8, 'isa': 66, 'jer': 52,
    'lam': 5, 'ezk': 48, 'dan': 12, 'hos': 14, 'jol': 3, 'amo': 9, 'oba': 1, 'jon': 4,
    'mic': 7, 'nam': 3, 'hab': 3, 'zep': 3, 'hag': 2, 'zec': 14, 'mal': 4, 'mat': 28,
    'mrk': 16, 'luk': 24, 'jhn': 21, 'act': 28, 'rom': 16, '1co': 16, '2co': 13, 'gal': 6,
    'eph': 6, 'php': 4, 'col': 4, '1th': 5, '2th': 3, '1ti': 6, '2ti': 4, 'tit': 3, 'phm': 1,
    'heb': 13, 'jas': 5, '1pe': 5, '2pe': 3, '1jo': 5, '2jn': 1, '3jn': 1, 'jud': 1, 'rev': 22
}


# Scrape the Bible verses
final_data = scrape_bible_verses(book_names, chapters)

# Save data to DataFrame
df = pd.DataFrame(final_data, columns=["Book", "Chapter", "Verse", "Text"])

# Save the DataFrame to an Excel file
excel_file = "bible_scraped_data.xlsx"
df.to_excel(excel_file, index=False)

# Download the file in Google Colab
from google.colab import files
files.download(excel_file)

print("Scraping Complete. Data saved and downloaded as Excel.")

NameError: name 'chapters' is not defined

In [ ]:
# Install required libraries
!pip install requests beautifulsoup4 pandas openpyxl

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from google.colab import files  # For downloading the file in Google Colab

def scrape_bible_excel(book_chapters, versions):
    base_url = "https://www.bible.com/bible/"
    all_verses = []

    for version, version_id in versions.items():
        for book, max_chapter in book_chapters.items():
            for chapter in range(1, max_chapter + 1):
                try:
                    url = f"{base_url}{version_id}/{book}.{chapter}"
                    print(f"Scraping: {url} ({version})")

                    response = requests.get(url)
                    if response.status_code != 200:
                        print(f"Failed to retrieve {book} {chapter} ({version}). Skipping...")
                        continue

                    soup = BeautifulSoup(response.content, 'html.parser')
                    verses = soup.find_all('span', class_='ChapterContent_verse__57FIw')

                    if not verses:
                        print(f"No verses found for {book} {chapter} ({version}). Skipping...")
                        continue

                    verse_data = {}
                    last_verse_num = None

                    for verse in verses:
                        verse_num_tag = verse.find('span', class_='ChapterContent_label__R2PLt')
                        verse_num = verse_num_tag.text.strip() if verse_num_tag else last_verse_num  # Keep track of last seen verse number

                        # Collect text parts (handling multi-line)
                        verse_text_parts = verse.find_all('span', class_='ChapterContent_content__RrUqA')
                        verse_text = " ".join([part.text.strip() for part in verse_text_parts if part.text.strip()])

                        # Merge with previous verse text if the same verse is spread over multiple spans
                        if verse_num in verse_data:
                            verse_data[verse_num] += " " + verse_text
                        else:
                            verse_data[verse_num] = verse_text

                        last_verse_num = verse_num  # Update last seen verse number

                    # Convert dictionary to list of rows
                    for verse_num, verse_text in verse_data.items():
                        all_verses.append({
                            "Version": version,
                            "Book": book.upper(),
                            "Chapter": chapter,
                            "Verse": verse_num,
                            "Text": verse_text
                        })

                    time.sleep(1)  # Delay to avoid being blocked

                except Exception as e:
                    print(f"Error processing {book} {chapter} ({version}): {e}. Skipping...")

    # Save data to DataFrame
    df = pd.DataFrame(all_verses)

    # Save the DataFrame to an Excel file
    excel_file = "RSV-C.xlsx"
    df.to_excel(excel_file, index=False, engine='openpyxl')

    # Download the file in Google Colab
    files.download(excel_file)

    print(f"RSV-C Bible verses have been saved to '{excel_file}' and downloaded to your computer.")

# Dictionary of books and their chapter counts (shortened for quick testing)
book_chapters = {
    "gen": 50, "exo": 40, "lev": 27, "num": 36, "deu": 34, "jos": 24, "jdg": 21, "rut": 4,
    "1sa": 31, "2sa": 24, "1ki": 22, "2ki": 25, "1ch": 29, "2ch": 36, "ezr": 10, "neh": 13,
    "est": 10, "job": 42, "psa": 150, "pro": 31, "ecc": 12, "sng": 8, "isa": 66, "jer": 52,
    "lam": 5, "ezk": 48, "dan": 12, "hos": 14, "jol": 3, "amo": 9, "oba": 1, "jon": 4,
    "mic": 7, "nam": 3, "hab": 3, "zep": 3, "hag": 2, "zec": 14, "mal": 4, "mat": 28, "mrk": 16,
    "luk": 24, "jhn": 21, "act": 28, "rom": 16, "1co": 16, "2co": 13, "gal": 6, "eph": 6,
    "php": 4, "col": 4, "1th": 5, "2th": 3, "1ti": 6, "2ti": 4, "tit": 3, "phm": 1, "heb": 13,
    "jas": 5, "1pe": 5, "2pe": 3, "1jo": 5, "2jn": 1, "3jn": 1, "jud": 1, "rev": 22
}

# Dictionary of Bible versions with their respective Bible.com IDs
versions = {
    "RSV-C": "2017",
}

# Call the function
scrape_bible_excel(book_chapters, versions)


Scraping: https://www.bible.com/bible/2017/gen.1 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.2 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.3 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.4 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.5 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.6 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.7 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.8 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.9 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.10 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.11 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.12 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.13 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.14 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.15 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.16 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.17 (RSV-C)
Scraping: https://www.b

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

RSV-C Bible verses have been saved to 'RSV-C.xlsx' and downloaded to your computer.


In [ ]:
# Install required libraries
!pip install requests beautifulsoup4 pandas openpyxl

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from google.colab import files  # For downloading the file in Google Colab

def scrape_bible_excel(book_chapters, versions):
    base_url = "https://www.bible.com/bible/"
    all_verses = []

    for version, version_id in versions.items():
        for book, max_chapter in book_chapters.items():
            for chapter in range(1, max_chapter + 1):
                try:
                    url = f"{base_url}{version_id}/{book}.{chapter}"
                    print(f"Scraping: {url} ({version})")

                    response = requests.get(url)
                    if response.status_code != 200:
                        print(f"Failed to retrieve {book} {chapter} ({version}). Skipping...")
                        continue

                    soup = BeautifulSoup(response.content, 'html.parser')
                    verses = soup.find_all('span', class_='ChapterContent_verse__57FIw')

                    if not verses:
                        print(f"No verses found for {book} {chapter} ({version}). Skipping...")
                        continue

                    verse_data = {}
                    last_verse_num = None

                    for verse in verses:
                        verse_num_tag = verse.find('span', class_='ChapterContent_label__R2PLt')
                        verse_num = verse_num_tag.text.strip() if verse_num_tag else last_verse_num  # Keep track of last seen verse number

                        # Collect text parts (handling multi-line)
                        verse_text_parts = verse.find_all('span', class_='ChapterContent_content__RrUqA')
                        verse_text = " ".join([part.text.strip() for part in verse_text_parts if part.text.strip()])

                        # Merge with previous verse text if the same verse is spread over multiple spans
                        if verse_num in verse_data:
                            verse_data[verse_num] += " " + verse_text
                        else:
                            verse_data[verse_num] = verse_text

                        last_verse_num = verse_num  # Update last seen verse number

                    # Convert dictionary to list of rows
                    for verse_num, verse_text in verse_data.items():
                        all_verses.append({
                            "Version": version,
                            "Book": book.upper(),
                            "Chapter": chapter,
                            "Verse": verse_num,
                            "Text": verse_text
                        })

                    time.sleep(1)  # Delay to avoid being blocked

                except Exception as e:
                    print(f"Error processing {book} {chapter} ({version}): {e}. Skipping...")

    # Save data to DataFrame
    df = pd.DataFrame(all_verses)

    # Save the DataFrame to an Excel file
    excel_file = "RV1895.xlsx"
    df.to_excel(excel_file, index=False, engine='openpyxl')

    # Download the file in Google Colab
    files.download(excel_file)

    print(f"RV1895 Bible verses have been saved to '{excel_file}' and downloaded to your computer.")

# Dictionary of books and their chapter counts (shortened for quick testing)
book_chapters = {
    "gen": 50, "exo": 40, "lev": 27, "num": 36, "deu": 34, "jos": 24, "jdg": 21, "rut": 4,
    "1sa": 31, "2sa": 24, "1ki": 22, "2ki": 25, "1ch": 29, "2ch": 36, "ezr": 10, "neh": 13,
    "est": 10, "job": 42, "psa": 150, "pro": 31, "ecc": 12, "sng": 8, "isa": 66, "jer": 52,
    "lam": 5, "ezk": 48, "dan": 12, "hos": 14, "jol": 3, "amo": 9, "oba": 1, "jon": 4,
    "mic": 7, "nam": 3, "hab": 3, "zep": 3, "hag": 2, "zec": 14, "mal": 4, "mat": 28, "mrk": 16,
    "luk": 24, "jhn": 21, "act": 28, "rom": 16, "1co": 16, "2co": 13, "gal": 6, "eph": 6,
    "php": 4, "col": 4, "1th": 5, "2th": 3, "1ti": 6, "2ti": 4, "tit": 3, "phm": 1, "heb": 13,
    "jas": 5, "1pe": 5, "2pe": 3, "1jn": 5, "2jn": 1, "3jn": 1, "jud": 1, "rev": 22
}

# Dictionary of Bible versions with their respective Bible.com IDs
versions = {
    "RV1895": "1922",
}

# Call the function
scrape_bible_excel(book_chapters, versions)

Scraping: https://www.bible.com/bible/1922/gen.1 (RV1895)
Scraping: https://www.bible.com/bible/1922/gen.2 (RV1895)
Scraping: https://www.bible.com/bible/1922/gen.3 (RV1895)
Scraping: https://www.bible.com/bible/1922/gen.4 (RV1895)
Scraping: https://www.bible.com/bible/1922/gen.5 (RV1895)
Scraping: https://www.bible.com/bible/1922/gen.6 (RV1895)
Scraping: https://www.bible.com/bible/1922/gen.7 (RV1895)
Scraping: https://www.bible.com/bible/1922/gen.8 (RV1895)
Scraping: https://www.bible.com/bible/1922/gen.9 (RV1895)
Scraping: https://www.bible.com/bible/1922/gen.10 (RV1895)
Scraping: https://www.bible.com/bible/1922/gen.11 (RV1895)
Scraping: https://www.bible.com/bible/1922/gen.12 (RV1895)
Scraping: https://www.bible.com/bible/1922/gen.13 (RV1895)
Scraping: https://www.bible.com/bible/1922/gen.14 (RV1895)
Scraping: https://www.bible.com/bible/1922/gen.15 (RV1895)
Scraping: https://www.bible.com/bible/1922/gen.16 (RV1895)
Scraping: https://www.bible.com/bible/1922/gen.17 (RV1895)
Scrapi

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

RV1895 Bible verses have been saved to 'RV1895.xlsx' and downloaded to your computer.


In [ ]:
# Install required libraries
!pip install requests beautifulsoup4 pandas openpyxl

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from google.colab import files  # For downloading the file in Google Colab

def scrape_bible_excel(book_chapters, versions):
    base_url = "https://www.bible.com/bible/"
    all_verses = []

    for version, version_id in versions.items():
        for book, max_chapter in book_chapters.items():
            for chapter in range(1, max_chapter + 1):
                try:
                    url = f"{base_url}{version_id}/{book}.{chapter}"
                    print(f"Scraping: {url} ({version})")

                    response = requests.get(url)
                    if response.status_code != 200:
                        print(f"Failed to retrieve {book} {chapter} ({version}). Skipping...")
                        continue

                    soup = BeautifulSoup(response.content, 'html.parser')
                    verses = soup.find_all('span', class_='ChapterContent_verse__57FIw')

                    if not verses:
                        print(f"No verses found for {book} {chapter} ({version}). Skipping...")
                        continue

                    verse_data = {}
                    last_verse_num = None

                    for verse in verses:
                        verse_num_tag = verse.find('span', class_='ChapterContent_label__R2PLt')
                        verse_num = verse_num_tag.text.strip() if verse_num_tag else last_verse_num  # Keep track of last seen verse number

                        # Collect text parts (handling multi-line)
                        verse_text_parts = verse.find_all('span', class_='ChapterContent_content__RrUqA')
                        verse_text = " ".join([part.text.strip() for part in verse_text_parts if part.text.strip()])

                        # Merge with previous verse text if the same verse is spread over multiple spans
                        if verse_num in verse_data:
                            verse_data[verse_num] += " " + verse_text
                        else:
                            verse_data[verse_num] = verse_text

                        last_verse_num = verse_num  # Update last seen verse number

                    # Convert dictionary to list of rows
                    for verse_num, verse_text in verse_data.items():
                        all_verses.append({
                            "Version": version,
                            "Book": book.upper(),
                            "Chapter": chapter,
                            "Verse": verse_num,
                            "Text": verse_text
                        })

                    time.sleep(1)  # Delay to avoid being blocked

                except Exception as e:
                    print(f"Error processing {book} {chapter} ({version}): {e}. Skipping...")

    # Save data to DataFrame
    df = pd.DataFrame(all_verses)

    # Save the DataFrame to an Excel file
    excel_file = "RSV-C.xlsx"
    df.to_excel(excel_file, index=False, engine='openpyxl')

    # Download the file in Google Colab
    files.download(excel_file)

    print(f"RSV-C Bible verses have been saved to '{excel_file}' and downloaded to your computer.")

# Dictionary of books and their chapter counts (shortened for quick testing)
book_chapters = {
    "gen": 50, "exo": 40, "lev": 27, "num": 36, "deu": 34, "jos": 24, "jdg": 21, "rut": 4,
    "1sa": 31, "2sa": 24, "1ki": 22, "2ki": 25, "1ch": 29, "2ch": 36, "ezr": 10, "neh": 13,
    "est": 10, "job": 42, "psa": 150, "pro": 31, "ecc": 12, "sng": 8, "isa": 66, "jer": 52,
    "lam": 5, "ezk": 48, "dan": 12, "hos": 14, "jol": 3, "amo": 9, "oba": 1, "jon": 4,
    "mic": 7, "nam": 3, "hab": 3, "zep": 3, "hag": 2, "zec": 14, "mal": 4, "mat": 28, "mrk": 16,
    "luk": 24, "jhn": 21, "act": 28, "rom": 16, "1co": 16, "2co": 13, "gal": 6, "eph": 6,
    "php": 4, "col": 4, "1th": 5, "2th": 3, "1ti": 6, "2ti": 4, "tit": 3, "phm": 1, "heb": 13,
    "jas": 5, "1pe": 5, "2pe": 3, "1jn": 5, "2jn": 1, "3jn": 1, "jud": 1, "rev": 22
}

# Dictionary of Bible versions with their respective Bible.com IDs
versions = {
    "RSV-C": "2017",
}

# Call the function
scrape_bible_excel(book_chapters, versions)

Scraping: https://www.bible.com/bible/2017/gen.1 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.2 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.3 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.4 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.5 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.6 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.7 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.8 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.9 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.10 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.11 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.12 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.13 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.14 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.15 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.16 (RSV-C)
Scraping: https://www.bible.com/bible/2017/gen.17 (RSV-C)
Scraping: https://www.b

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

RSV-C Bible verses have been saved to 'RSV-C.xlsx' and downloaded to your computer.


In [ ]:
# Install required libraries
!pip install requests beautifulsoup4 pandas openpyxl

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from google.colab import files  # For downloading the file in Google Colab

def scrape_bible_excel(book_chapters, versions):
    base_url = "https://www.bible.com/bible/"
    all_verses = []

    for version, version_id in versions.items():
        for book, max_chapter in book_chapters.items():
            for chapter in range(1, max_chapter + 1):
                try:
                    url = f"{base_url}{version_id}/{book}.{chapter}"
                    print(f"Scraping: {url} ({version})")

                    response = requests.get(url)
                    if response.status_code != 200:
                        print(f"Failed to retrieve {book} {chapter} ({version}). Skipping...")
                        continue

                    soup = BeautifulSoup(response.content, 'html.parser')
                    verses = soup.find_all('span', class_='ChapterContent_verse__57FIw')

                    if not verses:
                        print(f"No verses found for {book} {chapter} ({version}). Skipping...")
                        continue

                    verse_data = {}
                    last_verse_num = None

                    for verse in verses:
                        verse_num_tag = verse.find('span', class_='ChapterContent_label__R2PLt')
                        verse_num = verse_num_tag.text.strip() if verse_num_tag else last_verse_num  # Keep track of last seen verse number

                        # Collect text parts (handling multi-line)
                        verse_text_parts = verse.find_all('span', class_='ChapterContent_content__RrUqA')
                        verse_text = " ".join([part.text.strip() for part in verse_text_parts if part.text.strip()])

                        # Merge with previous verse text if the same verse is spread over multiple spans
                        if verse_num in verse_data:
                            verse_data[verse_num] += " " + verse_text
                        else:
                            verse_data[verse_num] = verse_text

                        last_verse_num = verse_num  # Update last seen verse number

                    # Convert dictionary to list of rows
                    for verse_num, verse_text in verse_data.items():
                        all_verses.append({
                            "Version": version,
                            "Book": book.upper(),
                            "Chapter": chapter,
                            "Verse": verse_num,
                            "Text": verse_text
                        })

                    time.sleep(1)  # Delay to avoid being blocked

                except Exception as e:
                    print(f"Error processing {book} {chapter} ({version}): {e}. Skipping...")

    # Save data to DataFrame
    df = pd.DataFrame(all_verses)

    # Save the DataFrame to an Excel file
    excel_file = "TEG.xlsx"
    df.to_excel(excel_file, index=False, engine='openpyxl')

    # Download the file in Google Colab
    files.download(excel_file)

    print(f"TEG Bible verses have been saved to '{excel_file}' and downloaded to your computer.")

# Dictionary of books and their chapter counts (shortened for quick testing)
book_chapters = {
    "gen": 50, "exo": 40, "lev": 27, "num": 36, "deu": 34, "jos": 24, "jdg": 21, "rut": 4,
    "1sa": 31, "2sa": 24, "1ki": 22, "2ki": 25, "1ch": 29, "2ch": 36, "ezr": 10, "neh": 13,
    "est": 10, "job": 42, "psa": 150, "pro": 31, "ecc": 12, "sng": 8, "isa": 66, "jer": 52,
    "lam": 5, "ezk": 48, "dan": 12, "hos": 14, "jol": 3, "amo": 9, "oba": 1, "jon": 4,
    "mic": 7, "nam": 3, "hab": 3, "zep": 3, "hag": 2, "zec": 14, "mal": 4, "mat": 28, "mrk": 16,
    "luk": 24, "jhn": 21, "act": 28, "rom": 16, "1co": 16, "2co": 13, "gal": 6, "eph": 6,
    "php": 4, "col": 4, "1th": 5, "2th": 3, "1ti": 6, "2ti": 4, "tit": 3, "phm": 1, "heb": 13,
    "jas": 5, "1pe": 5, "2pe": 3, "1jn": 5, "2jn": 1, "3jn": 1, "jud": 1, "rev": 22
}

# Dictionary of Bible versions with their respective Bible.com IDs
versions = {
    "TEG": "3010",
}

# Call the function
scrape_bible_excel(book_chapters, versions)

Scraping: https://www.bible.com/bible/3010/gen.1 (TEG)
No verses found for gen 1 (TEG). Skipping...
Scraping: https://www.bible.com/bible/3010/gen.2 (TEG)
No verses found for gen 2 (TEG). Skipping...
Scraping: https://www.bible.com/bible/3010/gen.3 (TEG)
No verses found for gen 3 (TEG). Skipping...
Scraping: https://www.bible.com/bible/3010/gen.4 (TEG)
No verses found for gen 4 (TEG). Skipping...
Scraping: https://www.bible.com/bible/3010/gen.5 (TEG)
No verses found for gen 5 (TEG). Skipping...
Scraping: https://www.bible.com/bible/3010/gen.6 (TEG)
No verses found for gen 6 (TEG). Skipping...
Scraping: https://www.bible.com/bible/3010/gen.7 (TEG)
No verses found for gen 7 (TEG). Skipping...
Scraping: https://www.bible.com/bible/3010/gen.8 (TEG)
No verses found for gen 8 (TEG). Skipping...
Scraping: https://www.bible.com/bible/3010/gen.9 (TEG)
No verses found for gen 9 (TEG). Skipping...
Scraping: https://www.bible.com/bible/3010/gen.10 (TEG)
No verses found for gen 10 (TEG). Skipping..

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

TEG Bible verses have been saved to 'TEG.xlsx' and downloaded to your computer.


In [ ]:
# Install required libraries
!pip install requests beautifulsoup4 pandas openpyxl

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from google.colab import files  # For downloading the file in Google Colab

def scrape_bible_excel(book_chapters, versions):
    base_url = "https://www.bible.com/bible/"
    all_verses = []

    for version, version_id in versions.items():
        for book, max_chapter in book_chapters.items():
            for chapter in range(1, max_chapter + 1):
                try:
                    url = f"{base_url}{version_id}/{book}.{chapter}"
                    print(f"Scraping: {url} ({version})")

                    response = requests.get(url)
                    if response.status_code != 200:
                        print(f"Failed to retrieve {book} {chapter} ({version}). Skipping...")
                        continue

                    soup = BeautifulSoup(response.content, 'html.parser')
                    verses = soup.find_all('span', class_='ChapterContent_verse__57FIw')

                    if not verses:
                        print(f"No verses found for {book} {chapter} ({version}). Skipping...")
                        continue

                    verse_data = {}
                    last_verse_num = None

                    for verse in verses:
                        verse_num_tag = verse.find('span', class_='ChapterContent_label__R2PLt')
                        verse_num = verse_num_tag.text.strip() if verse_num_tag else last_verse_num  # Keep track of last seen verse number

                        # Collect text parts (handling multi-line)
                        verse_text_parts = verse.find_all('span', class_='ChapterContent_content__RrUqA')
                        verse_text = " ".join([part.text.strip() for part in verse_text_parts if part.text.strip()])

                        # Merge with previous verse text if the same verse is spread over multiple spans
                        if verse_num in verse_data:
                            verse_data[verse_num] += " " + verse_text
                        else:
                            verse_data[verse_num] = verse_text

                        last_verse_num = verse_num  # Update last seen verse number

                    # Convert dictionary to list of rows
                    for verse_num, verse_text in verse_data.items():
                        all_verses.append({
                            "Version": version,
                            "Book": book.upper(),
                            "Chapter": chapter,
                            "Verse": verse_num,
                            "Text": verse_text
                        })

                    time.sleep(1)  # Delay to avoid being blocked

                except Exception as e:
                    print(f"Error processing {book} {chapter} ({version}): {e}. Skipping...")

    # Save data to DataFrame
    df = pd.DataFrame(all_verses)

    # Save the DataFrame to an Excel file
    excel_file = "TLV.xlsx"
    df.to_excel(excel_file, index=False, engine='openpyxl')

    # Download the file in Google Colab
    files.download(excel_file)

    print(f"TLV Bible verses have been saved to '{excel_file}' and downloaded to your computer.")

# Dictionary of books and their chapter counts (shortened for quick testing)
book_chapters = {
    "gen": 50, "exo": 40, "lev": 27, "num": 36, "deu": 34, "jos": 24, "jdg": 21, "rut": 4,
    "1sa": 31, "2sa": 24, "1ki": 22, "2ki": 25, "1ch": 29, "2ch": 36, "ezr": 10, "neh": 13,
    "est": 10, "job": 42, "psa": 150, "pro": 31, "ecc": 12, "sng": 8, "isa": 66, "jer": 52,
    "lam": 5, "ezk": 48, "dan": 12, "hos": 14, "jol": 3, "amo": 9, "oba": 1, "jon": 4,
    "mic": 7, "nam": 3, "hab": 3, "zep": 3, "hag": 2, "zec": 14, "mal": 4, "mat": 28, "mrk": 16,
    "luk": 24, "jhn": 21, "act": 28, "rom": 16, "1co": 16, "2co": 13, "gal": 6, "eph": 6,
    "php": 4, "col": 4, "1th": 5, "2th": 3, "1ti": 6, "2ti": 4, "tit": 3, "phm": 1, "heb": 13,
    "jas": 5, "1pe": 5, "2pe": 3, "1jn": 5, "2jn": 1, "3jn": 1, "jud": 1, "rev": 22
}

# Dictionary of Bible versions with their respective Bible.com IDs
versions = {
    "TLV": "314",
}

# Call the function
scrape_bible_excel(book_chapters, versions)

Scraping: https://www.bible.com/bible/314/gen.1 (TLV)
Scraping: https://www.bible.com/bible/314/gen.2 (TLV)
Scraping: https://www.bible.com/bible/314/gen.3 (TLV)
Scraping: https://www.bible.com/bible/314/gen.4 (TLV)
Scraping: https://www.bible.com/bible/314/gen.5 (TLV)
Scraping: https://www.bible.com/bible/314/gen.6 (TLV)
Scraping: https://www.bible.com/bible/314/gen.7 (TLV)
Scraping: https://www.bible.com/bible/314/gen.8 (TLV)
Scraping: https://www.bible.com/bible/314/gen.9 (TLV)
Scraping: https://www.bible.com/bible/314/gen.10 (TLV)
Scraping: https://www.bible.com/bible/314/gen.11 (TLV)
Scraping: https://www.bible.com/bible/314/gen.12 (TLV)
Scraping: https://www.bible.com/bible/314/gen.13 (TLV)
Scraping: https://www.bible.com/bible/314/gen.14 (TLV)
Scraping: https://www.bible.com/bible/314/gen.15 (TLV)
Scraping: https://www.bible.com/bible/314/gen.16 (TLV)
Scraping: https://www.bible.com/bible/314/gen.17 (TLV)
Scraping: https://www.bible.com/bible/314/gen.18 (TLV)
Scraping: https://w

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

TLV Bible verses have been saved to 'TLV.xlsx' and downloaded to your computer.


In [ ]:
# Install required libraries
!pip install requests beautifulsoup4 pandas openpyxl

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from google.colab import files  # For downloading the file in Google Colab

def scrape_bible_excel(book_chapters, versions):
    base_url = "https://www.bible.com/bible/"
    all_verses = []

    for version, version_id in versions.items():
        for book, max_chapter in book_chapters.items():
            for chapter in range(1, max_chapter + 1):
                try:
                    url = f"{base_url}{version_id}/{book}.{chapter}"
                    print(f"Scraping: {url} ({version})")

                    response = requests.get(url)
                    if response.status_code != 200:
                        print(f"Failed to retrieve {book} {chapter} ({version}). Skipping...")
                        continue

                    soup = BeautifulSoup(response.content, 'html.parser')
                    verses = soup.find_all('span', class_='ChapterContent_verse__57FIw')

                    if not verses:
                        print(f"No verses found for {book} {chapter} ({version}). Skipping...")
                        continue

                    verse_data = {}
                    last_verse_num = None

                    for verse in verses:
                        verse_num_tag = verse.find('span', class_='ChapterContent_label__R2PLt')
                        verse_num = verse_num_tag.text.strip() if verse_num_tag else last_verse_num  # Keep track of last seen verse number

                        # Collect text parts (handling multi-line)
                        verse_text_parts = verse.find_all('span', class_='ChapterContent_content__RrUqA')
                        verse_text = " ".join([part.text.strip() for part in verse_text_parts if part.text.strip()])

                        # Merge with previous verse text if the same verse is spread over multiple spans
                        if verse_num in verse_data:
                            verse_data[verse_num] += " " + verse_text
                        else:
                            verse_data[verse_num] = verse_text

                        last_verse_num = verse_num  # Update last seen verse number

                    # Convert dictionary to list of rows
                    for verse_num, verse_text in verse_data.items():
                        all_verses.append({
                            "Version": version,
                            "Book": book.upper(),
                            "Chapter": chapter,
                            "Verse": verse_num,
                            "Text": verse_text
                        })

                    time.sleep(1)  # Delay to avoid being blocked

                except Exception as e:
                    print(f"Error processing {book} {chapter} ({version}): {e}. Skipping...")

    # Save data to DataFrame
    df = pd.DataFrame(all_verses)

    # Save the DataFrame to an Excel file
    excel_file = "TOJB2011.xlsx"
    df.to_excel(excel_file, index=False, engine='openpyxl')

    # Download the file in Google Colab
    files.download(excel_file)

    print(f"TOJB2011 Bible verses have been saved to '{excel_file}' and downloaded to your computer.")

# Dictionary of books and their chapter counts (shortened for quick testing)
book_chapters = {
    "gen": 50, "exo": 40, "lev": 27, "num": 36, "deu": 34, "jos": 24, "jdg": 21, "rut": 4,
    "1sa": 31, "2sa": 24, "1ki": 22, "2ki": 25, "1ch": 29, "2ch": 36, "ezr": 10, "neh": 13,
    "est": 10, "job": 42, "psa": 150, "pro": 31, "ecc": 12, "sng": 8, "isa": 66, "jer": 52,
    "lam": 5, "ezk": 48, "dan": 12, "hos": 14, "jol": 3, "amo": 9, "oba": 1, "jon": 4,
    "mic": 7, "nam": 3, "hab": 3, "zep": 3, "hag": 2, "zec": 14, "mal": 4, "mat": 28, "mrk": 16,
    "luk": 24, "jhn": 21, "act": 28, "rom": 16, "1co": 16, "2co": 13, "gal": 6, "eph": 6,
    "php": 4, "col": 4, "1th": 5, "2th": 3, "1ti": 6, "2ti": 4, "tit": 3, "phm": 1, "heb": 13,
    "jas": 5, "1pe": 5, "2pe": 3, "1jn": 5, "2jn": 1, "3jn": 1, "jud": 1, "rev": 22
}

# Dictionary of Bible versions with their respective Bible.com IDs
versions = {
    "TOJB2011": "130",
}

# Call the function
scrape_bible_excel(book_chapters, versions)

Scraping: https://www.bible.com/bible/130/gen.1 (TOJB2011)
Scraping: https://www.bible.com/bible/130/gen.2 (TOJB2011)
Scraping: https://www.bible.com/bible/130/gen.3 (TOJB2011)
Scraping: https://www.bible.com/bible/130/gen.4 (TOJB2011)
Scraping: https://www.bible.com/bible/130/gen.5 (TOJB2011)
Scraping: https://www.bible.com/bible/130/gen.6 (TOJB2011)
Scraping: https://www.bible.com/bible/130/gen.7 (TOJB2011)
Scraping: https://www.bible.com/bible/130/gen.8 (TOJB2011)
Scraping: https://www.bible.com/bible/130/gen.9 (TOJB2011)
Scraping: https://www.bible.com/bible/130/gen.10 (TOJB2011)
Scraping: https://www.bible.com/bible/130/gen.11 (TOJB2011)
Scraping: https://www.bible.com/bible/130/gen.12 (TOJB2011)
Scraping: https://www.bible.com/bible/130/gen.13 (TOJB2011)
Scraping: https://www.bible.com/bible/130/gen.14 (TOJB2011)
Scraping: https://www.bible.com/bible/130/gen.15 (TOJB2011)
Scraping: https://www.bible.com/bible/130/gen.16 (TOJB2011)
Scraping: https://www.bible.com/bible/130/gen.17 

In [ ]:
# Install required libraries
!pip install requests beautifulsoup4 pandas openpyxl

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from google.colab import files  # For downloading the file in Google Colab

def scrape_bible_excel(book_chapters, versions):
    base_url = "https://www.bible.com/bible/"
    all_verses = []

    for version, version_id in versions.items():
        for book, max_chapter in book_chapters.items():
            for chapter in range(1, max_chapter + 1):
                try:
                    url = f"{base_url}{version_id}/{book}.{chapter}"
                    print(f"Scraping: {url} ({version})")

                    response = requests.get(url)
                    if response.status_code != 200:
                        print(f"Failed to retrieve {book} {chapter} ({version}). Skipping...")
                        continue

                    soup = BeautifulSoup(response.content, 'html.parser')
                    verses = soup.find_all('span', class_='ChapterContent_verse__57FIw')

                    if not verses:
                        print(f"No verses found for {book} {chapter} ({version}). Skipping...")
                        continue

                    verse_data = {}
                    last_verse_num = None

                    for verse in verses:
                        verse_num_tag = verse.find('span', class_='ChapterContent_label__R2PLt')
                        verse_num = verse_num_tag.text.strip() if verse_num_tag else last_verse_num  # Keep track of last seen verse number

                        # Collect text parts (handling multi-line)
                        verse_text_parts = verse.find_all('span', class_='ChapterContent_content__RrUqA')
                        verse_text = " ".join([part.text.strip() for part in verse_text_parts if part.text.strip()])

                        # Merge with previous verse text if the same verse is spread over multiple spans
                        if verse_num in verse_data:
                            verse_data[verse_num] += " " + verse_text
                        else:
                            verse_data[verse_num] = verse_text

                        last_verse_num = verse_num  # Update last seen verse number

                    # Convert dictionary to list of rows
                    for verse_num, verse_text in verse_data.items():
                        all_verses.append({
                            "Version": version,
                            "Book": book.upper(),
                            "Chapter": chapter,
                            "Verse": verse_num,
                            "Text": verse_text
                        })

                    time.sleep(1)  # Delay to avoid being blocked

                except Exception as e:
                    print(f"Error processing {book} {chapter} ({version}): {e}. Skipping...")

    # Save data to DataFrame
    df = pd.DataFrame(all_verses)

    # Save the DataFrame to an Excel file
    excel_file = "WBMS.xlsx"
    df.to_excel(excel_file, index=False, engine='openpyxl')

    # Download the file in Google Colab
    files.download(excel_file)

    print(f"WBMS Bible verses have been saved to '{excel_file}' and downloaded to your computer.")

# Dictionary of books and their chapter counts (shortened for quick testing)
book_chapters = {
    "gen": 50, "exo": 40, "lev": 27, "num": 36, "deu": 34, "jos": 24, "jdg": 21, "rut": 4,
    "1sa": 31, "2sa": 24, "1ki": 22, "2ki": 25, "1ch": 29, "2ch": 36, "ezr": 10, "neh": 13,
    "est": 10, "job": 42, "psa": 150, "pro": 31, "ecc": 12, "sng": 8, "isa": 66, "jer": 52,
    "lam": 5, "ezk": 48, "dan": 12, "hos": 14, "jol": 3, "amo": 9, "oba": 1, "jon": 4,
    "mic": 7, "nam": 3, "hab": 3, "zep": 3, "hag": 2, "zec": 14, "mal": 4, "mat": 28, "mrk": 16,
    "luk": 24, "jhn": 21, "act": 28, "rom": 16, "1co": 16, "2co": 13, "gal": 6, "eph": 6,
    "php": 4, "col": 4, "1th": 5, "2th": 3, "1ti": 6, "2ti": 4, "tit": 3, "phm": 1, "heb": 13,
    "jas": 5, "1pe": 5, "2pe": 3, "1jn": 5, "2jn": 1, "3jn": 1, "jud": 1, "rev": 22
}

# Dictionary of Bible versions with their respective Bible.com IDs
versions = {
    "WBMS": "2407",
}

# Call the function
scrape_bible_excel(book_chapters, versions)